# Projeto 2 - Ciência dos Dados

##### Integrantes
    Júlia Ribeiro Martins
    Igor Figueiredo

___
# Classificador automático de sentimento


### Introdução

## Preparando o ambiente no jupyter:

In [209]:
%%capture

#Instalando o tweepy, arquivo que permite contato entre a plataforma Twitter e Python
!pip install tweepy

In [210]:
#importando as bibliotecas

import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import re 
%matplotlib inline
import numpy as np
import math as math
from scipy import stats
import matplotlib.pyplot as plt
import os
import seaborn as sns

___
## Autenticando no  Twitter

* Conta: *** @IJ_Projeto_2***

In [211]:
#identificador da conta no twitter: @IJ_Projeto_2

#Dados de autenticação do twitter:
#leitura do arquivo no formato JSON.
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca.
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

In [212]:
# Definindo o diretório a ser trabalhado.
os.chdir("/Users/Figueiredo/Documents/Insper/2º semestre/Ciência dos dados/Projetos/Projeto 2")
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/Figueiredo/Documents/Insper/2º semestre/Ciência dos dados/Projetos/Projeto 2


___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens

De forma a analisar se as pessoas gostaram da série '13 Reasons Why', selecionaremos tal produto para avaliar se os comentários postados no twitter revela a opinião do público, podendo este ser contra ou a favor à serie; ou até mesmo irrelevante para a classificação definida.

In [213]:
#Produto escolhido:
produto = '13 reasons why'

#Quantidade mínima de mensagens capturadas:
n = 950
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [214]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [215]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens

Uma vez que a tabela do excel foi gerada, é necessario realizar a avaliação manual de cada um dos tweets, classificando-os assim de 1 para aqueles referente as pessoas que gostaram da série, 2 para quem não gostou da série e 3 para os comentários irrelevantes de acordo com a classificação definida acima.

Retirando os retweets da tabela de Treinamento, inicialmente contendo 650 tweets

In [216]:
tabela_treinamento_sujo        = pd.read_excel("13 reasons why_classificacao.xlsx") 
tabela_treinamento_sujo_index  = tabela_treinamento_sujo.set_index('tr')        
tabela_t_sujo_index            = tabela_treinamento_sujo_index[['Treinamento']][:]
sem_rt_treino                  = tabela_t_sujo_index.loc[[1,2,3]]
tabelarelevancia               = tabela_treinamento_sujo[['Treinamento','tr']][:]

Retirando os retweets da tabela de Teste, inicialmente com 300 tweets

In [217]:
t               = pd.read_excel("13 reasons why_classificacao_teste.xlsx")
te              = t.set_index('te')
tes             = te[['Teste']][:]                  
sem_rt_teste    = tes.loc[[1,2,3]]
tabela_de_teste = sem_rt_teste.reset_index()

Limpando os tweets da tabela para montar o classificador

In [218]:
gostou      = sem_rt_treino.loc[[1]]
naogostou   = sem_rt_treino.loc[[2]]
irrelevante = sem_rt_treino.loc[[3]]

In [219]:
# Criando função para limpar os carcteres especiais.
novoindex=[]
def cleanup(text):
    punctuation = '[\n•!\-.,:?");=+(█]' 
    pattern = re.compile(punctuation)
    for frase in text:
        novoindex.append(re.sub(pattern,'',frase))
    return novoindex

# Adcionando o resultado da função no nosso DataFrame.
novo=cleanup(sem_rt_treino["Treinamento"])
sem_rt_treino['limpo']= novo
# sem_rt_treino

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


Separando o data frame de acordo com cada classificação selecionada acima: Gostou( 1 ), Não gostou( 2 ) e Irrelevante( 3 )

In [220]:
gostou_limpo      = sem_rt_treino.loc[[1]]
nao_gostou_limpo  = sem_rt_treino.loc[[2]]
irrelevante_limpo = sem_rt_treino.loc[[3]]

Tabela de frequências 

In [221]:
#tabela de frequencia absoluta das pessoas que gostaram, nao gostaram + comentarios irrelevantes.
frequencia_absoluta = pd.Series(np.sum(sem_rt_treino.limpo).split()).value_counts() 

# frequencia_absoluta

Calculando a frenquência relativa para cada data frame: Gostou( 1 ), Não gostou( 2 ) e Irrelevante( 3 )

In [222]:
i = pd.Series(np.sum(irrelevante_limpo.limpo).split()) #Separa as palavras em strings
# i.value_counts().head()


j = pd.Series(np.sum(nao_gostou_limpo.limpo).split()) #Separa as palavras em strings
# j.value_counts().head()


k = pd.Series(np.sum(gostou_limpo.limpo).split()) #Separa as palavras em strings
# k.value_counts().head()

In [223]:
#Freqência relativa do número de palavras em Irrelevante
relativo_irrelevante = i.value_counts(True)
# relativo_irrelevante.head()

In [224]:
#Freqência relativa do número de palavras em não gostou
relativo_não_gostou = j.value_counts(True)
# relativo_não_gostou.head()

In [225]:
#Freqência relativa do número de palavras em gostou
relativo_gostou = k.value_counts(True)
# relativo_gostou.head()

Definindo a porcentagem de uma frase ser "Gostou", "Não gostou" ou "Irrelevante"

In [226]:
#Redefinindo o index dos tweets treinamento
treino_reindex = sem_rt_treino.reset_index()
# treino_reindex.head()

In [227]:
coluna_relevância = treino_reindex.tr #Cria uma coluna com as relevâncias.

#Calculando a quantidade de frases de acordo com cada classificação feita manualmente.
total_por_classificação = coluna_relevância.value_counts()
# total_por_classificação

In [228]:
prob_gosto       = total_por_classificação[1]/len(coluna_relevância) #Calcula a provabilidade de uma palavra ser relevante entre todas as palavras do DataFrame
# prob_relevante
prob_nao_gosto   = total_por_classificação[2]/len(coluna_relevância) #Calcula a provabilidade de uma palavra ser irrelevante entre todas as palavras do DataFrame
prob_irrelevante = total_por_classificação[3]/len(coluna_relevância) #Calcula a provabilidade de uma palavra ser irrelevante entre todas as palavras do DataFrame
print('A probabilidade de ser gosto é       = ',int(prob_gosto * 100),'%')
print('A probabilidade de ser não gosto é   = ',int(prob_nao_gosto * 100),'%')
print('A probabilidade de ser irrelevante é = ', int(prob_irrelevante * 100),'%')

A probabilidade de ser gosto é       =  29 %
A probabilidade de ser não gosto é   =  34 %
A probabilidade de ser irrelevante é =  36 %


___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [233]:
gost = np.sum(gostou_limpo.limpo).split()
all_gost = len(gost)

n_gost = np.sum(nao_gostou_limpo.limpo).split()
all_n_gost = len(n_gost)

irrelev = np.sum(irrelevante_limpo.limpo).split()
all_irrelev = len(irrelev)

Tudo = all_gost + all_n_gost + all_irrelev

def Naive(Tweet):
    naive = Tweet.split(' ')
    gosta = []
    n_gosta = []
    irrelevante = []
    for i in range(len(naive)):
        gosta.append(np.log((gost.count(naive[i]) + 1) / (all_gost + Tudo)))
        n_gosta.append(np.log((n_gost.count(naive[i]) + 1) / (all_n_gost + Tudo)))
        irrelevante.append(np.log((irrelev.count(naive[i]) + 1) / (all_irrelev + Tudo)))
    P_gost = np.sum(gosta)
    P_n_gost = np.sum(n_gosta)
    P_irrelev = np.sum(irrelevante)
    
    #Aplicando o log para 
    p1 = np.log(prob_gosto) + P_gost
    p2 = np.log(prob_nao_gosto) + P_n_gost
    p3 = np.log(prob_irrelevante) + P_irrelev
    
    if p1 > p2 and p1 > p3:
        return 1
    elif p2 > p1 and p2 > p3:
        return 2
    elif p3 > p1 and p3 > p2:
        return 3
    

In [230]:
lista_teste = []


for e in tabela_de_teste.Teste:
    lista_teste.append(Naive(e))
    
tabela_de_teste["Classificação-Naive"] = lista_teste
tabela_de_teste

# tabela_de_teste[tabela_de_teste['Classificação-Naive']==1]

,te,Teste,Classificação-Naive
0,1,minha amg tá nervosa pq eu amo o \nbryce walke...,2
1,1,finamente terminei a 3° temp de 13 reasons why...,3
2,1,"essa série é massa ""13 reasons why""",2
3,1,"13 reasons why,a série que todo mundo critica ...",3
4,1,atribuí nota 9 ao episódio 3x1 - yeah. i'm the...,1
5,1,• sobre séries •\n\n1. homeland\n2. stranger t...,3
6,1,"hmmm terminei 13 reasons why, até q gostei",1
7,1,@netflixbrasil eu vendo a 1 temp de 13 reasons...,3
8,1,manoooo assisti o trailer da 3 temporada de 13...,3
9,1,ao contrário de tds as críticas sobre a 3 temp...,3


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

In [ ]:
Fazer grafico de pizza para concluir
# adim = dados.EDUCACAO[dados.DEFAULT == 'Adimplente'].value_counts()
# adim.plot(kind='pie', autopct="%.2f",legend=False)
# plt.title('ADIMPLENTES')

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**